In [1]:
import pandas as pd

In [62]:
febMar20 = pd.read_csv('Simmer_FebMar20.csv')

/Users/pranavkrishnan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,10,11,12,15,16,17,28,34,41,46,47,52,53,54,58,63,66,67,72,73,74,75,76,77,78,79,80,81,82,83,87,88,89,94,95,96,97,98,99,100,102,108,109,112,113,115,116,117,118,119,121,124,125,126,127,131,132,133,134,135,136,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
febMar20.head()

,name,distinct_id,time,sampling_factor,dataset,properties.$ae_session_length,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,...,properties.distance,properties.ratingCount,properties.Name,properties.CollectionName,properties.IsFoursquareRestaurant,properties.Restaurant,properties.searchQuery,properties.Rated,properties.restaurantComment,properties.ListName
0,$ae_session,fac4dd756360cc15,1582560733000,1,$mixpanel,20.3,101091,101091,1.4.7,1.4.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AppOpen,fac4dd756360cc15,1582560712000,1,$mixpanel,NaN,101091,101091,1.4.7,1.4.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$ae_session,B16482B3-A3CD-4A27-B579-34F79CC47396,1583524264000,1,$mixpanel,12.8,2.5.0,2.5.0,2.5.0,2.5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$ae_session,B16482B3-A3CD-4A27-B579-34F79CC47396,1583524376000,1,$mixpanel,106.3,2.5.0,2.5.0,2.5.0,2.5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AppOpen,B16482B3-A3CD-4A27-B579-34F79CC47396,1583524243000,1,$mixpanel,NaN,2.5.0,2.5.0,2.5.0,2.5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
augSep19 = pd.read_csv('Simmer_AugSep19.csv')

/Users/pranavkrishnan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32,33,34,35,36,37,38,40,41,44,45,54,81,83,84,85,86,87,94,95,96,97,98,99,100,101,102,103,104,110,114,115,116,118,119,120,121,122,123,124,127,131,133,134,135,140,144,148,152,156,160,164,168,170,171,172,173,174,175,176,177,180,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
augSep19.head()

,name,distinct_id,time,sampling_factor,dataset,properties.$ae_updated_version,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,...,properties.initialCategoriesSelected.12.Selected,properties.initialCategoriesSelected.13.Id,properties.initialCategoriesSelected.13.MenuId,properties.initialCategoriesSelected.13.Name,properties.initialCategoriesSelected.13.Selected,properties.initialCategoriesSelected.14.Id,properties.initialCategoriesSelected.14.MenuId,properties.initialCategoriesSelected.14.Name,properties.initialCategoriesSelected.14.Selected,properties.searchQuery
0,$ae_updated,4A47A27E-992F-40D8-8A7E-08CA601D441A,1565013412000,1,$mixpanel,1.9.1,1.9.1,1.9.1,1.9.1,1.9.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AppOpen,4A47A27E-992F-40D8-8A7E-08CA601D441A,1565013413000,1,$mixpanel,NaN,1.9.1,1.9.1,1.9.1,1.9.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AppOpen,866F0631-15CB-40CD-8E0A-04753AFFB01A,1565145944000,1,$mixpanel,NaN,1.9.2,1.9.2,1.9.2,1.9.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Visit Signup/Login Page,866F0631-15CB-40CD-8E0A-04753AFFB01A,1565145948000,1,$mixpanel,NaN,1.9.2,1.9.2,1.9.2,1.9.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$ae_first_open,12C9B87A-53B0-4283-9083-40AE9F8AB95A,1565522925000,1,$mixpanel,NaN,1.9.3,1.9.3,1.9.3,1.9.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
decJan19 = pd.read_csv('Simmer_DecJan19.csv')

/Users/pranavkrishnan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,13,20,26,33,34,36,37,38,39,40,42,43,44,45,47,64,70,72,73,75,76,77,79,80,81,82,89,90,91,92,93,97,98,99,101,105,106,107,109,110,111,112,114,117,118,119,120,124,128,132,136,140,144,148,152,156,160,164,166,167,168,173,177,181,185,189,193,195,196,197,199,202,205,209,213,217,221,225,229,233,235,237,239,240) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
decJan19.head()

,name,distinct_id,time,sampling_factor,dataset,properties.$ae_session_length,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,...,properties.initialCategoriesSelected.11.Id,properties.initialCategoriesSelected.11.MenuId,properties.initialCategoriesSelected.11.Name,properties.initialCategoriesSelected.11.Selected,properties.$browser,properties.$browser_version,properties.$device,properties.campaign_id,properties.existingDate,properties.newDate
0,$ae_session,29152811-7C4C-43AE-B829-D467F9073B0F,1580447805000,1,$mixpanel,59,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AppOpen,29152811-7C4C-43AE-B829-D467F9073B0F,1580447746000,1,$mixpanel,NaN,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feed - Scroll Down,29152811-7C4C-43AE-B829-D467F9073B0F,1580447753000,1,$mixpanel,NaN,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Feed - Scroll Down,29152811-7C4C-43AE-B829-D467F9073B0F,1580447772000,1,$mixpanel,NaN,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Home - Explore Nearby,29152811-7C4C-43AE-B829-D467F9073B0F,1580447750000,1,$mixpanel,NaN,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
octNov19 = pd.read_csv('Simmer_OctNov19.csv')

/Users/pranavkrishnan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,35,36,37,38,42,43,44,45,78,80,81,82,83,84,85,87,88,89,90,91,92,93,94,95,104,111,112,113,115,116,123,124,125,126,127,128,129,131,132,133,134,135,136,139,140,141,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,260,262,263,264,266,268,270,271,273,274,275) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
octNov19.head()

,name,distinct_id,time,sampling_factor,dataset,properties.$ae_session_length,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,...,properties.ItemName,properties.CategoryId,properties.CategoryName,properties.ItemId,properties.existingDate,properties.newDate,properties.restaurantDescription,properties.Follow,properties.userMessage,properties.Page
0,$ae_session,D007B42B-0927-46EE-A333-AF997781510F,1575129207000,1,$mixpanel,10,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$ae_session,D007B42B-0927-46EE-A333-AF997781510F,1575129265000,1,$mixpanel,22.6,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$ae_session,D007B42B-0927-46EE-A333-AF997781510F,1575129278000,1,$mixpanel,10.7,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$ae_session,D007B42B-0927-46EE-A333-AF997781510F,1575129353000,1,$mixpanel,63.7,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$ae_session,D007B42B-0927-46EE-A333-AF997781510F,1575129953000,1,$mixpanel,596.2,2.3.3,2.3.3,2.3.3,2.3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
nullCountsAugSep = pd.DataFrame({'Column Name': augSep19.columns, 'Null Count' : [augSep19[column].isnull().sum() for column in augSep19.columns]})


In [58]:
nullCountsDecJan = pd.DataFrame({'Column Name': decJan19.columns, 'Null Count' : [decJan19[column].isnull().sum() for column in decJan19.columns]})


In [65]:
nullCountsFebMar = pd.DataFrame({'Column Name': febMar20.columns, 'Null Count' : [febMar20[column].isnull().sum() for column in febMar20.columns]})


In [73]:
nullCountsOctNov = pd.DataFrame({'Column Name': octNov19.columns, 'Null Count': [octNov19[column].isnull().sum() for column in octNov19.columns]})


In [82]:
nullCountsAugSep[nullCountsAugSep['Null Count'] < 0.3 * len(augSep19)]


,Column Name,Null Count
0,name,0
1,distinct_id,0
2,time,0
3,sampling_factor,0
4,dataset,0
6,properties.$app_build_number,0
7,properties.$app_release,0
8,properties.$app_version,0
9,properties.$app_version_string,0
10,properties.$carrier,1181


In [61]:
nullCountsDecJan[nullCountsDecJan['Null Count'] < .3 * len(decJan19)]

,Column Name,Null Count
0,name,0
1,distinct_id,0
2,time,0
3,sampling_factor,0
4,dataset,0
6,properties.$app_build_number,1
7,properties.$app_release,1
8,properties.$app_version,1
9,properties.$app_version_string,1
10,properties.$carrier,551


In [80]:
nullCountsFebMar[nullCountsFebMar['Null Count'] < .2 * len(febMar20)]

,Column Name,Null Count
0,name,0
1,distinct_id,0
2,time,0
3,sampling_factor,0
4,dataset,0
6,properties.$app_build_number,0
7,properties.$app_release,0
8,properties.$app_version,0
9,properties.$app_version_string,0
13,properties.$carrier,414


In [77]:
nullCountsOctNov[nullCountsOctNov['Null Count'] < .3 * len(octNov19)]

,Column Name,Null Count
0,name,0
1,distinct_id,0
2,time,0
3,sampling_factor,0
4,dataset,0
6,properties.$app_build_number,1050
7,properties.$app_release,1050
8,properties.$app_version,1050
9,properties.$app_version_string,1050
10,properties.$carrier,1711


In [95]:
columns_values = {}
df_list = [febMar20, decJan19, octNov19, augSep19]
selected_columns = list(nullCountsOctNov[nullCountsOctNov['Null Count'] < .3 * len(octNov19)]['Column Name'])
for column in selected_columns:
    for df in df_list:
        if column in columns_values:
            columns_values[column] += list(df[column])
        else:
            columns_values[column] = []
            columns_values[column] += list(df[column])

past7Months = pd.DataFrame(columns_values)

In [96]:
past7Months

,name,distinct_id,time,sampling_factor,dataset,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,properties.$carrier,...,properties.$region,properties.$screen_height,properties.$screen_width,properties.$user_id,properties.$wifi,properties.UUId,properties.mp_country_code,properties.mp_device_model,properties.mp_lib,properties.mp_processing_time_ms
0,$ae_session,fac4dd756360cc15,1582560733000,1,$mixpanel,101091,101091,1.4.7,1.4.7,Google Fi,...,California,2094.0,1080.0,NaN,True,fac4dd756360cc15,US,NaN,android,1582589534045
1,AppOpen,fac4dd756360cc15,1582560712000,1,$mixpanel,101091,101091,1.4.7,1.4.7,Google Fi,...,California,2094.0,1080.0,NaN,True,fac4dd756360cc15,US,NaN,android,1582589533972
2,$ae_session,B16482B3-A3CD-4A27-B579-34F79CC47396,1583524264000,1,$mixpanel,2.5.0,2.5.0,2.5.0,2.5.0,T-Mobile,...,Maryland,812.0,375.0,B16482B3-A3CD-4A27-B579-34F79CC47396,False,B16482B3-A3CD-4A27-B579-34F79CC47396,US,"iPhone12,3",iphone,1583553129756
3,$ae_session,B16482B3-A3CD-4A27-B579-34F79CC47396,1583524376000,1,$mixpanel,2.5.0,2.5.0,2.5.0,2.5.0,T-Mobile,...,Maryland,812.0,375.0,B16482B3-A3CD-4A27-B579-34F79CC47396,False,B16482B3-A3CD-4A27-B579-34F79CC47396,US,"iPhone12,3",iphone,1583553176144
4,AppOpen,B16482B3-A3CD-4A27-B579-34F79CC47396,1583524243000,1,$mixpanel,2.5.0,2.5.0,2.5.0,2.5.0,T-Mobile,...,Maryland,812.0,375.0,B16482B3-A3CD-4A27-B579-34F79CC47396,False,B16482B3-A3CD-4A27-B579-34F79CC47396,US,"iPhone12,3",iphone,1583553129756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190366,Onboarding - Allow Location,A8E48126-0505-4560-AF2E-58720466A847,1569787552000,1,$mixpanel,2.1.6,2.1.6,2.1.6,2.1.6,AT&T,...,California,812.0,375.0,A8E48126-0505-4560-AF2E-58720466A847,True,A8E48126-0505-4560-AF2E-58720466A847,US,"iPhone12,3",iphone,1569812761064
190367,Onboarding - Allow Location,A8E48126-0505-4560-AF2E-58720466A847,1569787554000,1,$mixpanel,2.1.6,2.1.6,2.1.6,2.1.6,AT&T,...,California,812.0,375.0,A8E48126-0505-4560-AF2E-58720466A847,True,A8E48126-0505-4560-AF2E-58720466A847,US,"iPhone12,3",iphone,1569812761031
190368,Onboarding - Allow Location,A8E48126-0505-4560-AF2E-58720466A847,1569787555000,1,$mixpanel,2.1.6,2.1.6,2.1.6,2.1.6,AT&T,...,California,812.0,375.0,A8E48126-0505-4560-AF2E-58720466A847,True,A8E48126-0505-4560-AF2E-58720466A847,US,"iPhone12,3",iphone,1569812761031
190369,Onboarding - Allow Location,A8E48126-0505-4560-AF2E-58720466A847,1569787559000,1,$mixpanel,2.1.6,2.1.6,2.1.6,2.1.6,AT&T,...,California,812.0,375.0,A8E48126-0505-4560-AF2E-58720466A847,True,A8E48126-0505-4560-AF2E-58720466A847,US,"iPhone12,3",iphone,1569812761030
